# Problem 2: Demand Estimation

The sales of two companies, Company 1 and Company 2, in two regions,
Region 1 and Region 2, are provided here. The description of the only three columns
are:

    ● Sales_U → Sales in equivalent units (lbs)
    ● Sales_USD → Sales in \$
    ● date → starting date for the week


● Construct time-series plots of sales and prices for Company 1 in Region 1 and 2. Repeat the exercise for Company 2. Describe the differences or similarities between Company 1 and 2 pricing policies.